In [1]:
import pickle
import sys
sys.path.append("../")
from Join_scheme.data_prepare import process_stats_data
from BayesCard.Models.Bayescard_BN import Bayescard_BN
import time
import pandas as pd
import numpy as np
from BayesCard.Evaluation.cardinality_estimation import parse_query_single_table

In [2]:
from Join_scheme.data_prepare import process_stats_data
data_path = "/home/ubuntu/End-to-End-CardEst-Benchmark/datasets/stats_simplified/{}.csv"
model_folder = "/home/ubuntu/data_CE/saved_models"
data, null_values, key_attrs, table_buckets, equivalent_keys, schema, bin_size = process_stats_data(data_path,
                                                model_folder, 200, "sub_optimal", get_bin_means=True)

bucketizing equivalent key group: {'postLinks.PostId', 'posts.Id', 'comments.PostId', 'votes.PostId', 'postLinks.RelatedPostId', 'postHistory.PostId', 'tags.ExcerptPostId'}
bucketizing equivalent key group: {'badges.UserId', 'comments.UserId', 'postHistory.UserId', 'users.Id', 'votes.UserId', 'posts.OwnerUserId'}


In [6]:
import numpy as np
import copy

from Join_scheme.join_graph import process_condition, get_join_hyper_graph
from Join_scheme.data_prepare import identify_key_values
from BayesCard.Evaluation.cardinality_estimation import timestamp_transorform, construct_table_query


class Factor:
    """
    This the class defines a multidimensional conditional probability.
    """
    def __init__(self, variables, pdfs, equivalent_variables=[]):
        self.variables = variables
        self.equivalent_variables = equivalent_variables
        self.pdfs = pdfs
        self.cardinalities = dict()
        for i, var in enumerate(self.variables):
            self.cardinalities[var] = pdfs.shape[i]
            if len(equivalent_variables) != 0:
                self.cardinalities[equivalent_variables[i]] = pdfs.shape[i]


class Bound_ensemble:
    """
    This the class where we store all the trained models and perform inference on the bound.
    """
    def __init__(self, bns, all_bin_means, schema):
        self.bns = bns
        self.table_buckets = all_bin_means
        self.schema = schema
        self.all_keys, self.equivalent_keys = identify_key_values(schema)

    def parse_query_simple(self, query):
        """
        If your selection query contains no aggregation and nested sub-queries, you can use this function to parse a
        join query. Otherwise, use parse_query function.
        """
        query = query.replace(" where ", " WHERE ")
        query = query.replace(" from ", " FROM ")
        query = query.replace(" and ", " AND ")
        query = query.split(";")[0]
        query = query.strip()
        tables_all = {}
        join_cond = []
        table_query = {}
        join_keys = {}
        tables_str = query.split(" WHERE ")[0].split(" FROM ")[-1]
        for table_str in tables_str.split(","):
            table_str = table_str.strip()
            if " as " in table_str:
                tables_all[table_str.split(" as ")[-1]] = table_str.split(" as ")[0]
            else:
                tables_all[table_str.split(" ")[-1]] = table_str.split(" ")[0]

        # processing conditions
        conditions = query.split(" WHERE ")[-1].split(" AND ")
        for cond in conditions:
            table, cond, join, join_key = process_condition(cond, tables_all)
            if not join:
                attr = cond[0]
                op = cond[1]
                value = cond[2]
                if "Date" in attr:
                    assert "::timestamp" in value
                    value = timestamp_transorform(value.strip().split("::timestamp")[0])
                if table not in table_query:
                    table_query[table] = dict()
                construct_table_query(self.bns[table], table_query[table], attr, op, value)
            else:
                join_cond.append(cond)
                for tab in join_key:
                    if tab in join_keys:
                        join_keys[tab].add(join_key[tab])
                    else:
                        join_keys[tab] = set([join_key[tab]])

        return tables_all, table_query, join_cond, join_keys

    def get_all_id_conidtional_distribution(self, table_queries, join_keys, equivalent_group):
        res = dict()
        for table in join_keys:
            key_attrs = list(join_keys[table])
            if table in table_queries:
                table_query = table_queries[table]
            else:
                table_query = {}
            id_attrs, probs = self.bns[table].query_id_prob(table_query, key_attrs)
            new_id_attrs = []
            for K in id_attrs:
                for PK in equivalent_group:
                    if K in equivalent_group[PK]:
                        new_id_attrs.append(PK)
            assert len(new_id_attrs) == len(id_attrs)
            res[table] = Factor(id_attrs, probs, new_id_attrs)
        return res

    def eliminate_one_key_group(self, tables, key_group, factors, relevant_keys):
        """This version only supports 2D distributions"""
        rest_group = None
        rest_group_cardinalty = 0
        eliminated_tables = []
        rest_group_tables = []
        for table in tables:
            assert key_group in factors[table].equivalent_variables
            temp = copy.deepcopy(factors[table].equivalent_variables)
            temp.remove(key_group)
            if len(temp) == 0:
                eliminated_tables.append(table)
            for key in temp:
                if rest_group:
                    assert factors[table].cardinalities[key] == rest_group_cardinalty
                    rest_group_tables.append(table)
                else:
                    rest_group = key
                    rest_group_cardinalty = factors[table].cardinalities[key]
                    rest_group_tables = [table]

        all_probs_eliminated = []
        all_modes_eliminated = []
        for table in eliminated_tables:
            bin_modes = self.table_buckets[table].oned_bin_modes[relevant_keys[key_group][table]]
            all_probs_eliminated.append(factors[table].pdfs)
            all_modes_eliminated.append(np.minimum(bin_modes, factors[table].pdfs))
        if rest_group:
            new_factor_pdf = np.zeros(rest_group_cardinalty)
        else:
            return self.compute_bound_oned(all_probs_eliminated, all_modes_eliminated)

        for i in range(rest_group_cardinalty):
            rest_group_probs_eliminated = []
            rest_group_modes_eliminated = []
            for table in rest_group_tables:

                idx_f = factors[table].equivalent_variables.index(key_group)
                idx_b = self.table_buckets[table].id_attributes.index(relevant_keys[key_group][table])
                bin_modes = self.table_buckets[table].twod_bin_modes[relevant_keys[key_group][table]]
                if idx_f == 0 and idx_b == 0:
                    rest_group_probs_eliminated.append(factors[table].pdfs[:, i])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[:, i]))
                elif idx_f == 0 and idx_b == 1:
                    rest_group_probs_eliminated.append(factors[table].pdfs[:, i])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[:, i]))
                elif idx_f == 1 and idx_b == 0:
                    rest_group_probs_eliminated.append(factors[table].pdfs[i, :])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[:, i], factors[table].pdfs[i, :]))
                else:
                    rest_group_probs_eliminated.append(factors[table].pdfs[i, :])
                    rest_group_modes_eliminated.append(np.minimum(bin_modes[i, :], factors[table].pdfs[i, :]))
            new_factor_pdf[i] = self.compute_bound_oned(all_probs_eliminated + rest_group_probs_eliminated,
                                                        all_modes_eliminated + rest_group_modes_eliminated)

        for table in rest_group_tables:
            factors[table] = Factor([rest_group], new_factor_pdf, [rest_group])

        return None

    def compute_bound_oned(self, all_probs, all_modes):
        all_probs = np.stack(all_probs, axis=0)
        all_modes = np.stack(all_modes, axis=0)
        multiplier = np.prod(all_modes, axis=0)
        non_zero_idx = np.where(multiplier != 0)[0]
        min_number = np.amin(all_probs[:, non_zero_idx]/all_modes[:, non_zero_idx], axis=0)
        multiplier[non_zero_idx] = multiplier[non_zero_idx] * min_number
        return np.sum(multiplier)

    def get_optimal_elimination_order(self, equivalent_group, join_keys, factors):
        cardinalities = dict()
        lengths = dict()
        tables_involved = dict()
        relevant_keys = dict()
        for group in equivalent_group:
            relevant_keys[group] = dict()
            lengths[group] = len(equivalent_group[group])
            cardinalities[group] = []
            tables_involved[group] = set([])
            for keys in equivalent_group[group]:
                for table in join_keys:
                    if keys in join_keys[table]:
                        cardinalities[group].append(len(join_keys[table]))
                        tables_involved[group].add(table)
                        variables = factors[table].variables
                        variables[variables.index(keys)] = group
                        factors[table].variables = variables
                        relevant_keys[group][table] = keys
                        break
            cardinalities[group] = np.asarray(cardinalities[group])

        optimal_order = list(equivalent_group.keys())
        for i in range(len(optimal_order)):
            min_idx = i
            for j in range(i+1, len(optimal_order)):
                min_group = optimal_order[min_idx]
                curr_group = optimal_order[j]
                if np.max(cardinalities[curr_group]) < np.max(cardinalities[min_group]):
                    min_idx = j
                else:
                    min_max_tables = np.max(cardinalities[min_group])
                    min_num_max_tables = len(np.where(cardinalities[min_group] == min_max_tables)[0])
                    curr_max_tables = np.max(cardinalities[curr_group])
                    curr_num_max_tables = len(np.where(cardinalities[curr_group] == curr_max_tables)[0])
                    if curr_num_max_tables < min_num_max_tables:
                        min_idx = j
                    elif lengths[curr_group] < lengths[min_group]:
                        min_idx = j
            optimal_order[i], optimal_order[min_idx] = optimal_order[min_idx], optimal_order[i]
        return optimal_order, tables_involved, relevant_keys

    def get_cardinality(self, query_str):
        tables_all, table_queries, join_cond, join_keys = self.parse_query_simple(query_str)
        equivalent_group = get_join_hyper_graph(join_keys, self.equivalent_keys)
        conditional_factors = self.get_all_id_conidtional_distribution(table_queries, join_keys, equivalent_group)
        optimal_order, tables_involved, relevant_keys = self.get_optimal_elimination_order(equivalent_group, join_keys,
                                                                            conditional_factors)

        for key_group in optimal_order:
            tables = tables_involved[key_group]
            res = self.eliminate_one_key_group(tables, key_group, conditional_factors, relevant_keys)
        return res


In [7]:
model_path = "/home/ubuntu/data_CE/CE_scheme_models/model_stats_sub_optimal_200.pkl"
with open(model_path, "rb") as f:
    old_BE = pickle.load(f)

In [8]:
BE = Bound_ensemble(old_BE.bns, table_buckets, schema)


In [9]:
query_file = "/home/ubuntu/End-to-End-CardEst-Benchmark/workloads/stats_CEB/sub_plan_queries/stats_CEB_sub_queries.sql"
with open(query_file, "r") as f:
    queries = f.readlines()

In [10]:
qerror = []
latency = []
pred = []
for i, query_str in enumerate(queries):
    #if i == 10: break
    query = query_str.split("||")[0][:-1]
    print("========================")
    true_card = int(query_str.split("||")[-1])
    t = time.time()
    res = BE.get_cardinality(query)
    pred.append(res)
    latency.append(time.time() - t)
    qerror.append(res/true_card)
    print(f"estimating query {i}: predicted {res}, true_card {true_card}, qerror {res/true_card}, latency {time.time() - t}")

estimating query 0: predicted 79851.0, true_card 79851, qerror 1.0, latency 0.010972976684570312
estimating query 1: predicted 10063259.994332165, true_card 10220614, qerror 0.9846042512056677, latency 0.01441335678100586
estimating query 2: predicted 2028154.683314832, true_card 1458075, qerror 1.3909810423433855, latency 0.009134769439697266
estimating query 3: predicted 2090158.4490934496, true_card 1709781, qerror 1.2224714446431735, latency 0.010031938552856445
estimating query 4: predicted 7862228.866210119, true_card 7491903, qerror 1.0494301469479943, latency 0.009088993072509766
estimating query 5: predicted 511459.3807856937, true_card 428612, qerror 1.1932922568329718, latency 0.014743328094482422
estimating query 6: predicted 55077231.3043023, true_card 55900138, qerror 0.9852789863291983, latency 0.01362919807434082
estimating query 7: predicted 11102.0, true_card 10972, qerror 1.0118483412322274, latency 0.010178804397583008
estimating query 8: predicted 55606146.84610309

estimating query 58: predicted 33619.21739606571, true_card 31965, qerror 1.051750896169739, latency 0.014846563339233398
estimating query 59: predicted 33295.71565782266, true_card 26836, qerror 1.2407108234395088, latency 0.03501772880554199
estimating query 60: predicted 2868194.78798813, true_card 2704241, qerror 1.06062839369277, latency 0.00802755355834961
estimating query 61: predicted 33661.61931326377, true_card 32918, qerror 1.022590051438841, latency 0.007128238677978516
estimating query 62: predicted 83987.69229481785, true_card 86112, qerror 0.9753308748469186, latency 0.008111000061035156
estimating query 63: predicted 2513414.650419802, true_card 2488080, qerror 1.0101824098983159, latency 0.010039091110229492
estimating query 64: predicted 1011783.6739766671, true_card 1056687, qerror 0.957505556495601, latency 0.004668474197387695
estimating query 65: predicted 27569.269956384032, true_card 20334, qerror 1.3558212824030704, latency 0.006531953811645508
estimating query

estimating query 120: predicted 34742.75, true_card 34789, qerror 0.9986705567851907, latency 0.003211498260498047
estimating query 121: predicted 1531430.7423328375, true_card 1486002, qerror 1.0305711178940793, latency 0.0072956085205078125
estimating query 122: predicted 36717.674915131, true_card 37048, qerror 0.9910838618854189, latency 0.0062255859375
estimating query 123: predicted 79851.0, true_card 79851, qerror 1.0, latency 0.0014984607696533203
estimating query 124: predicted 228646956.0682782, true_card 232039659, qerror 0.985378779876065, latency 0.008621454238891602
estimating query 125: predicted 1290801.2256178982, true_card 1187068, qerror 1.0873860853951907, latency 0.007435798645019531
estimating query 126: predicted 1011783.6739766671, true_card 1056687, qerror 0.957505556495601, latency 0.003275632858276367
estimating query 127: predicted 1531430.7423328375, true_card 1486002, qerror 1.0305711178940793, latency 0.00640559196472168
estimating query 128: predicted 22

estimating query 193: predicted 13774.653764443421, true_card 3236, qerror 4.256691521768672, latency 0.012064695358276367
estimating query 194: predicted 1586720.6158672115, true_card 52313, qerror 30.3312869815765, latency 0.013683319091796875
estimating query 195: predicted 37420.43970827328, true_card 35295, qerror 1.0602192862522535, latency 0.002440929412841797
estimating query 196: predicted 599141.638439791, true_card 642583, qerror 0.9323957192141576, latency 0.003996372222900391
estimating query 197: predicted 941857.7834182123, true_card 865125, qerror 1.0886956028530124, latency 0.0011947154998779297
estimating query 198: predicted 39198.000857922285, true_card 59346, qerror 0.6604994583952125, latency 0.0050051212310791016
estimating query 199: predicted 73475.4791807575, true_card 69701, qerror 1.0541524394306752, latency 0.0023937225341796875
estimating query 200: predicted 1115800.666040152, true_card 1057588, qerror 1.055042857937261, latency 0.003924131393432617
estim

estimating query 257: predicted 70726.43717435551, true_card 68676, qerror 1.0298566773597109, latency 0.008857965469360352
estimating query 258: predicted 708460.5498745213, true_card 404170, qerror 1.75287762544108, latency 0.012407302856445312
estimating query 259: predicted 61708562.25662128, true_card 58846511, qerror 1.0486358699604346, latency 0.006358146667480469
estimating query 260: predicted 384557.1201697689, true_card 264252, qerror 1.4552666400623986, latency 0.009384632110595703
estimating query 261: predicted 694329.3431041335, true_card 550289, qerror 1.2617539930911457, latency 0.011433124542236328
estimating query 262: predicted 22484379.88487083, true_card 7661811, qerror 2.9346038273289214, latency 0.012628793716430664
estimating query 263: predicted 2780480.08607542, true_card 1236745, qerror 2.248224238687377, latency 0.009929656982421875
estimating query 264: predicted 145658.29763304605, true_card 130349, qerror 1.117448523832527, latency 0.008943796157836914
e

...

estimating query 2306: predicted 5147530.171303263, true_card 413991602, qerror 0.01243389997873257, latency 0.011409282684326172
estimating query 2307: predicted 3403677.658840808, true_card 4154456, qerror 0.8192835978623454, latency 0.010105609893798828
estimating query 2308: predicted 15725034.029328026, true_card 70129559, qerror 0.22422833186970456, latency 0.013938665390014648
estimating query 2309: predicted 15728096.163108135, true_card 81958485, qerror 0.19190320761917615, latency 0.01139974594116211
estimating query 2310: predicted 646978.5449156475, true_card 797534, qerror 0.8112237784416056, latency 0.009992361068725586
estimating query 2311: predicted 2808822.748108315, true_card 2793773, qerror 1.0053868900974827, latency 0.010366201400756836
estimating query 2312: predicted 147661.05595303746, true_card 19108639, qerror 0.007727450183816726, latency 0.016663789749145508
estimating query 2313: predicted 147661.05595303746, true_card 168708, qerror 0.8752463188054951, la

estimating query 2365: predicted 202108.44294587348, true_card 121625, qerror 1.661734371600193, latency 0.013466119766235352
estimating query 2366: predicted 1462096.8261852104, true_card 488680, qerror 2.9919309695203617, latency 0.04768633842468262
estimating query 2367: predicted 43283.77736467535, true_card 1706513, qerror 0.025363872038874212, latency 0.013836860656738281
estimating query 2368: predicted 38213.29913770175, true_card 17903, qerror 2.134463449572795, latency 0.012850284576416016
estimating query 2369: predicted 2996618.456945235, true_card 1677253, qerror 1.786622803444224, latency 0.04908108711242676
estimating query 2370: predicted 15723488.005614247, true_card 31274384, qerror 0.5027593191160615, latency 0.018915414810180664
estimating query 2371: predicted 445929.49005828414, true_card 354404, qerror 1.258251853981005, latency 0.014397382736206055
estimating query 2372: predicted 754066.9652765439, true_card 21200677, qerror 0.03556806064620219, latency 0.05140

estimating query 2431: predicted 14866285.94481172, true_card 65723184, qerror 0.22619546163210413, latency 0.01714348793029785
estimating query 2432: predicted 573502.6320380222, true_card 686665, qerror 0.8352000350069133, latency 0.010302543640136719
estimating query 2433: predicted 14869004.68860948, true_card 61160701, qerror 0.2431137061134973, latency 0.022773027420043945
estimating query 2434: predicted 804631.4018124093, true_card 655992, qerror 1.226587217241078, latency 0.016256093978881836
estimating query 2435: predicted 14697007.909834787, true_card 14929017, qerror 0.984459185077945, latency 0.006103515625
estimating query 2436: predicted 44972.81346322869, true_card 38925, qerror 1.155370930333428, latency 0.0071070194244384766
estimating query 2437: predicted 268611.05897851055, true_card 65432, qerror 4.105194079021129, latency 0.01311492919921875
estimating query 2438: predicted 446346.3481620789, true_card 339466, qerror 1.3148484624736465, latency 0.014059543609619

estimating query 2491: predicted 65485.901224622525, true_card 64490, qerror 1.0154427232845793, latency 0.012010574340820312
estimating query 2492: predicted 491595.06087939464, true_card 520351, qerror 0.9447374193177195, latency 0.0050466060638427734
estimating query 2493: predicted 766256.8121264847, true_card 427774, qerror 1.791265509653426, latency 0.003162384033203125
estimating query 2494: predicted 1266984.7688529682, true_card 1035019, qerror 1.2241174015674767, latency 0.011228561401367188
estimating query 2495: predicted 127392.12395924213, true_card 77261, qerror 1.6488541949915498, latency 0.014328479766845703
estimating query 2496: predicted 4079102.1771812607, true_card 4255848, qerror 0.9584698929992943, latency 0.003351926803588867
estimating query 2497: predicted 847146.653511131, true_card 948354, qerror 0.89328104643533, latency 0.004875659942626953
estimating query 2498: predicted 1293335.5560644106, true_card 2410386, qerror 0.5365678177953285, latency 0.0120687

estimating query 2553: predicted 59791.07274870793, true_card 60963, qerror 0.9807764176419784, latency 0.020799875259399414
estimating query 2554: predicted 316478.3966504127, true_card 172157, qerror 1.8383126834831736, latency 0.020757675170898438
estimating query 2555: predicted 1129042.888435989, true_card 438690, qerror 2.5736690793863297, latency 0.01465153694152832
estimating query 2556: predicted 125800.45127838982, true_card 50824, qerror 2.475217442121632, latency 0.017251253128051758
estimating query 2557: predicted 234393.2511273831, true_card 249157, qerror 0.9407451973148782, latency 0.02118539810180664
estimating query 2558: predicted 1190488.4786561136, true_card 659522, qerror 1.8050777360817587, latency 0.021984577178955078
estimating query 2559: predicted 253206.30136793727, true_card 87188, qerror 2.9041416406837786, latency 0.023213863372802734
estimating query 2560: predicted 1031576.9397734762, true_card 436286, qerror 2.3644511622501665, latency 0.0228893756866

In [11]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(qerror, i)}")
print(f"total inference time: {np.sum(latency)}")

q-error 50% percentile is 1.0886956028530124
q-error 90% percentile is 8.997195827788868
q-error 95% percentile is 35.944280097940556
q-error 99% percentile is 325.3228824523701
q-error 100% percentile is 43600.54179062004
total inference time: 34.85584306716919


In [12]:
with open("stats_CEB_no_bound.txt", "w") as f:
    for p in pred:
        f.write(str(p)+"\n")